In [54]:
import spotipy
from spotipy import util
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
import pandas as pd
import matplotlib.pyplot as plt
import urllib

## REMOVE BEFORE PUSHING ##
CLIENT_ID='472796d4b7904eb8ab972808d46bd0b0'
CLIENT_SECRET='cc89036aabd04a61a9d20970a0510186'
LOCAL_REDIRECT_URI = 'http://127.0.0.1:8080'
REDIRECT_URI = 'https://fioto-spotify-flask.herokuapp.com/'
SCOPE = 'playlist-modify-public user-top-read'

username = 'elw86ve5g5t944wwlef6qyzu3'


modify_playlist = 'playlist-modify-public'
user_auth = 'user-top-read'

token = util.prompt_for_user_token(
                        client_id=CLIENT_ID,
                        client_secret=CLIENT_SECRET,
                        redirect_uri=LOCAL_REDIRECT_URI,
                        scope=SCOPE,
                        username=username)

sp = spotipy.Spotify(auth=token)


In [55]:
# Test Artists URIs
logslaught = 'spotify:artist:1I471vwcRhqQl6QonGZlen'
grandson = 'spotify:artist:4ZgQDCtRqZlhLswVS6MHN4'
run_the_jewels = 'spotify:artist:4RnBFZRiMLRyZy0AzzTg2C'
hatebreed = 'spotify:artist:17Mb968quDHpjCkIyq30QV'
jxdn = 'spotify:artist:6Y64EaNqpqcZYTgs4c76gF'

In [56]:
albums = sp.artist_albums(artist_id = run_the_jewels, limit=50)
len(albums['items'])

50

In [57]:
def get_artist_tracks(sp, artists, n_albums=50):
    '''
    Function takes in the Spotify URI of one or more artists and returns a Pandas dataframe with Spotify's proprietary audio features.
    
    Parameters:
    sp: Spotipy user authentication object. 
    artists: list of one or more artist URIs
    n_albums: number of albums to fetch per artist. 1-50 acceptable range
    
    
    '''
    # Empty list to hold the track uris, album uris and dataframes to concatenate
    uris = []
    album_uris = []
    df_list = []
    
    # Checking if user input artists as a list. 
    if type(artists) != list:
        # Rectifying if user did not input list
        artists = [artists]
    
    # Iterate through artists
    for artist in artists:
        # Fetches the albums JSONs
        albums = sp.artist_albums(artist, limit=n_albums)
        # Iterating through the albums and appending the artist URIs
        for album in albums['items']:
            album_uris.append(album['uri'])
    
    # Iterate through the album URIs
    for uri in album_uris:
        # Fetching album JSON
        album = sp.album(uri)
        # Fetching album name
        album_name = album['name']
        # Fetching all artists on album
        album_artist = ', '.join([artist['name'] for artist in album['artists']])
        
        # Fetching all of the track URIs on the album
        tracks = sp.album_tracks(uri)
        # Listing track URIs and names
        track_uris = [track['uri'] for track in tracks['items']]
        track_names = [track['name'] for track in tracks['items']]
        
        # Fetching audio features from the track URIs and assigning track name, album name and album artist
        audio_features_df = pd.DataFrame.from_dict(sp.audio_features(track_uris))
        audio_features_df['track_name'] = track_names
        audio_features_df['album_name'] = album_name
        audio_features_df['artist_name'] = album_artist
        
        # Appending dataframe to list to concatenate
        df_list.append(audio_features_df)
    
    # Concatenating dataframes, resetting index and dropping duplicates
    df = pd.concat(df_list)
    df.reset_index(inplace=True, drop=True)
    df.drop_duplicates()
    return df

In [58]:
art = get_artist_tracks(sp, 'spotify:artist:6Y64EaNqpqcZYTgs4c76gF')

In [59]:
art

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,type,id,uri,track_href,analysis_url,duration_ms,time_signature,track_name,album_name,artist_name
0,0.684,0.672,10,-5.762,0,0.0341,0.15100,0.000000,0.1160,0.381,...,audio_features,4ih3Y0t86lfK8m8pTgEx4I,spotify:track:4ih3Y0t86lfK8m8pTgEx4I,https://api.spotify.com/v1/tracks/4ih3Y0t86lfK...,https://api.spotify.com/v1/audio-analysis/4ih3...,155674,4,Better Off Dead,Better Off Dead,jxdn
1,0.501,0.633,5,-6.560,1,0.0466,0.00358,0.000000,0.3510,0.463,...,audio_features,4C7dLHxwVRA0cYNzqbA7aP,spotify:track:4C7dLHxwVRA0cYNzqbA7aP,https://api.spotify.com/v1/tracks/4C7dLHxwVRA0...,https://api.spotify.com/v1/audio-analysis/4C7d...,135573,4,Tonight (feat. iann dior),Better Off Dead,jxdn
2,0.552,0.333,0,-7.836,1,0.0270,0.08680,0.000000,0.5570,0.274,...,audio_features,1JMaSFk9pS772cWad0gp4F,spotify:track:1JMaSFk9pS772cWad0gp4F,https://api.spotify.com/v1/tracks/1JMaSFk9pS77...,https://api.spotify.com/v1/audio-analysis/1JMa...,119371,4,Pray,Better Off Dead,jxdn
3,0.598,0.662,4,-5.797,1,0.0480,0.00301,0.000167,0.0812,0.531,...,audio_features,0nAsW4plPTFBHoQkU1z6CA,spotify:track:0nAsW4plPTFBHoQkU1z6CA,https://api.spotify.com/v1/tracks/0nAsW4plPTFB...,https://api.spotify.com/v1/audio-analysis/0nAs...,141207,4,So What!,Better Off Dead,jxdn
4,0.554,0.800,8,-3.936,1,0.0395,0.11900,0.000000,0.6290,0.463,...,audio_features,3himokjS8OCS9Qfgzn1JAH,spotify:track:3himokjS8OCS9Qfgzn1JAH,https://api.spotify.com/v1/tracks/3himokjS8OCS...,https://api.spotify.com/v1/audio-analysis/3him...,160120,4,Angels & Demons,Better Off Dead,jxdn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,0.741,0.821,9,-5.186,1,0.0507,0.18200,0.000000,0.2630,0.161,...,audio_features,48pbmUwTtlSDpi3sRd3EzR,spotify:track:48pbmUwTtlSDpi3sRd3EzR,https://api.spotify.com/v1/tracks/48pbmUwTtlSD...,https://api.spotify.com/v1/audio-analysis/48pb...,182693,4,Replay (feat. Flo Rida),UK Hits,Various Artists
345,0.598,0.662,4,-5.797,1,0.0480,0.00301,0.000167,0.0812,0.531,...,audio_features,5LX9kAJoI2KarxvMZHFQtl,spotify:track:5LX9kAJoI2KarxvMZHFQtl,https://api.spotify.com/v1/tracks/5LX9kAJoI2Ka...,https://api.spotify.com/v1/audio-analysis/5LX9...,141207,4,So What!,UK Hits,Various Artists
346,0.920,0.610,9,-6.615,1,0.2910,0.00558,0.000000,0.0938,0.424,...,audio_features,5dqqWVQmrKDeNDd1a0yjxh,spotify:track:5dqqWVQmrKDeNDd1a0yjxh,https://api.spotify.com/v1/tracks/5dqqWVQmrKDe...,https://api.spotify.com/v1/audio-analysis/5dqq...,135493,4,Cool Off,UK Hits,Various Artists
347,0.692,0.653,5,-5.715,1,0.0392,0.49000,0.006560,0.4550,0.901,...,audio_features,4CkweyEaChnaHSNLaUFmSO,spotify:track:4CkweyEaChnaHSNLaUFmSO,https://api.spotify.com/v1/tracks/4CkweyEaChna...,https://api.spotify.com/v1/audio-analysis/4Ckw...,192267,4,Right Back Where We Started From,UK Hits,Various Artists


In [60]:
def get_user_tracks(sp, limit=10, time_range='medium_term'):
    '''
    Fetches a Spotify user's top tracks and returns a dataframe of 
    Spotify audio features
    
    Parameters:
    sp: Spotipy user authentication object  
    limit: number of user tracks to fetch
    time_range: choose from 'short_term', 'medium_term' and 'long_term'
    '''
    
    # Fetching user's top tracks
    user_tracks = sp.current_user_top_tracks(limit=limit, time_range=time_range)
    # Fetching audio features from user's top tracks
    audio_features =  sp.audio_features([item['uri'] for item in user_tracks['items']])
    # Listing names of tracks
    names = [item['name'] for item in user_tracks['items']]
    # Creating dataframe of audio features
    df = pd.DataFrame.from_dict(audio_features)
    # Setting track names
    df['track_name'] = names
    return df

In [61]:
def user_tracks_dataframe(spotify, limit=10):
    '''
    This function takes in a spotify user authenication object and a limit of songs.
    Return a pandas data frame with users top tracks (names and artists) 
    within short, medium and long-term time-ranges

    '''
    terms = ['short_term', 'medium_term', 'long_term']
    df_list = []
    for term in terms:
        tracks = spotify.current_user_top_tracks(limit=limit, time_range=term)
        track_names = [track['name'] for track in tracks['items']]
        artist_list = []
        for track in tracks['items']:
            artists = ', '.join([artist['name'] for artist in track['artists']])
            artist_list.append(artists)
        df = pd.DataFrame(list(zip(track_names, artist_list)), 
                          columns = ['track_name', 'artist_name'])
        df['time_range'] = term
        df_list.append(df)
    df = pd.concat(df_list)
    df.reset_index(inplace=True, drop=True)
    return df
        

In [62]:
def get_playlist_embeddings(spotify):
    '''
    Takes in user authentication object and returns HTML embedding codes for user playlists
    '''
    playlists = spotify.current_user_playlists()
    playlist_ids = [playlist['id'] for playlist in playlists['items']]
    embed_code = '<iframe src="https://open.spotify.com/embed/playlist/####" width="300" height="380" frameborder="0" allowtransparency="true" allow="encrypted-media"></iframe>'
    codes = ' '.join([embed_code.replace('####', playlist_id) for playlist_id in playlist_ids])
    return codes

In [63]:
%%html
'<iframe src="https://open.spotify.com/embed/playlist/2rX148mot1TpEDPbJdrqjs" width="300" height="380" frameborder="0" allowtransparency="true" allow="encrypted-media"></iframe> <iframe src="https://open.spotify.com/embed/playlist/36IMauEyDsOmc6NoJ1ECPM" width="300" height="380" frameborder="0" allowtransparency="true" allow="encrypted-media"></iframe> <iframe src="https://open.spotify.com/embed/playlist/7gX4gtuKBJyZS8qll6W6ud" width="300" height="380" frameborder="0" allowtransparency="true" allow="encrypted-media"></iframe> <iframe src="https://open.spotify.com/embed/playlist/04xlIANGeecMR5UNd8RBic" width="300" height="380" frameborder="0" allowtransparency="true" allow="encrypted-media"></iframe>'

In [64]:
from sklearn.metrics.pairwise import pairwise_distances, cosine_distances, cosine_similarity
pd.set_option("display.precision", 14)

In [69]:
def compare_songs(artist_df, user_df):
    '''
    Takes in a dataframe of artist tracks and audio feautes, and a dataframe of user top tracks. 
    Returns tracks in artist_df that are most alligned with user tracks.
    '''
    
    band_track_names = artist_df['track_name']
    user_track_names = user_df['track_name']
    
    df_numeric = artist_df.loc[:, artist_df.columns[:11]]
    user_df_numeric = user_df.loc[:, user_df.columns[:11]]
    
    recs = pairwise_distances(df_numeric, user_df_numeric, metric='cosine')
    rec_df = pd.DataFrame(recs, columns=user_track_names, index=band_track_names)    
    rec_df = 1 - rec_df
    rec_df.drop_duplicates(inplace=True)
    
    song_list = []
    
    for user_track in rec_df.columns:
        max_cosine = rec_df[user_track].max()
        song_name = rec_df[rec_df[user_track] == max_cosine].index[0]
        song_uri = list(artist_df.loc[artist_df['track_name'] == song_name, 'uri'])[0]
        #print(f'{user_track}------ closest to ---------> {song_name}')
        song_list.append((song_name, song_uri))
        
    uri_list = []
    [uri_list.append(track[1]) for track in song_list if track[1] not in uri_list]
    
    return uri_list

# Test out Functions!
get_artist_tracks
get_user_tracks
compare_songs

In [75]:
glass_animals = 'spotify:artist:4yvcSjfu4PC0CYQyLy4wSq'
ewf = 'spotify:artist:4QQgXkCYTt3BlENzhyNETg'

In [80]:
artist_tracks = get_artist_tracks(sp, artists = ewf )
user_tracks = get_user_tracks(sp, limit=12, time_range='long_term')

In [81]:
artist_tracks.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,type,id,uri,track_href,analysis_url,duration_ms,time_signature,track_name,album_name,artist_name
0,0.399,0.336,6,-15.191,0,0.1160,0.3690,0.00008950,0.136,0.0579,...,audio_features,52IRsLlMfFeLb0lIyRMV4Y,spotify:track:52IRsLlMfFeLb0lIyRMV4Y,https://api.spotify.com/v1/tracks/52IRsLlMfFeL...,https://api.spotify.com/v1/audio-analysis/52IR...,93413,4,Intro : Grandmix - Earth Wind & Fire And Friends,Grandmix Earth Wind & Fire (mixed by Ben Liebr...,"Earth, Wind & Fire, Friends"
1,0.644,0.813,4,-8.584,0,0.0558,0.2420,0.00117000,0.630,0.7740,...,audio_features,3C4G2ZtQBwqLHyS3yfAmYM,spotify:track:3C4G2ZtQBwqLHyS3yfAmYM,https://api.spotify.com/v1/tracks/3C4G2ZtQBwqL...,https://api.spotify.com/v1/audio-analysis/3C4G...,104693,4,Fantasy,Grandmix Earth Wind & Fire (mixed by Ben Liebr...,"Earth, Wind & Fire, Friends"
2,0.493,0.902,4,-7.602,0,0.0817,0.2820,0.00000568,0.115,0.5710,...,audio_features,2GuvbIb0MqNbEVlm9WjUVr,spotify:track:2GuvbIb0MqNbEVlm9WjUVr,https://api.spotify.com/v1/tracks/2GuvbIb0MqNb...,https://api.spotify.com/v1/audio-analysis/2Guv...,96427,4,Sun Goddess,Grandmix Earth Wind & Fire (mixed by Ben Liebr...,"Earth, Wind & Fire, Friends"
3,0.795,0.766,1,-8.047,0,0.0522,0.2590,0.00017800,0.141,0.9270,...,audio_features,2y5BrHSV5E0c2cEIBWyD83,spotify:track:2y5BrHSV5E0c2cEIBWyD83,https://api.spotify.com/v1/tracks/2y5BrHSV5E0c...,https://api.spotify.com/v1/audio-analysis/2y5B...,104053,4,On Your Face,Grandmix Earth Wind & Fire (mixed by Ben Liebr...,"Earth, Wind & Fire, Friends"
4,0.780,0.579,2,-6.960,0,0.0228,0.0404,0.00003010,0.316,0.8720,...,audio_features,2qOrEjVUu1Ww8LPvXd1OXA,spotify:track:2qOrEjVUu1Ww8LPvXd1OXA,https://api.spotify.com/v1/tracks/2qOrEjVUu1Ww...,https://api.spotify.com/v1/audio-analysis/2qOr...,97747,4,Wonderland,Grandmix Earth Wind & Fire (mixed by Ben Liebr...,"Earth, Wind & Fire, Friends"


In [89]:
song_list = compare_songs(artist_tracks, user_tracks)

In [454]:
tracks = get_artist_tracks(sp, artists='spotify:artist:3bcLBxvaI7GsBzGp3WHnwQ')
user_tracks = get_user_tracks(sp)

In [453]:
tracks

499

In [456]:
len(user_tracks)

10

In [459]:
compare_songs(tracks, user_tracks)

['spotify:track:2IF8sqTWsKptMllsYglkeK',
 'spotify:track:0POBjbE1ovNWInRDVQbREC',
 'spotify:track:7KsGeiA9C4x63MCJcdpL12',
 'spotify:track:1jIluyycudIsv7OrURzBBU',
 'spotify:track:0BhFrPI9q21LadxoIgT2zf',
 'spotify:track:76g5npBAaPI8RDQETVehR7',
 'spotify:track:0OkNhfAJPFb54zavGW0RAN',
 'spotify:track:4e2rhIyiyAeWuU23k0kMFC',
 'spotify:track:7DFTzN8Tg1U6zIYn8rgL9y']